## Set up

In [51]:
# Import Dependencies
import pandas as pd
import requests
import urllib.parse as UP
import yaml
import pymongo
import bs4
import re
import numpy as np
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [52]:
# Set up path for chromedriver

with open("config.yml", "r") as ymlpath:
    config = yaml.safe_load(ymlpath)
    executable_path = {"executable_path": config["config-key"]}

## FOR TESTING PURPOSES: For windows, un-comment cell below:

In [3]:
# executable_path = {"executable_path": "chromedriver"}

## Scraping Box Office Mojo 

In [53]:
def movie_scraper():
    
    """Scrapes www.boxofficemojo.com for the top ten movies for 2018 based on gross box-office amount.
    Returns a list of dictionaries with year, rank, movie title, and studio"""
    
    year = str(2018)
    movie_df_list=[]
    
    # Get webpage data using requests
    response = requests.get("https://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr=%s&p=.htm" % year)
    
    #Parse HTML using Beautiful Soup
    soup = bs(response.text,"html.parser")

    # Find location of necessary data in soup object
    soup_tables = soup.find_all("table")
    soup_elements = soup_tables[3].find_all("td")

    # For each td element, find and store data in a list 
    movie_data=[]

    for i in soup_elements:
        if i.find("a")!=None:
            movie_data.append(i.find("a").contents[0]) 
        elif i.find("font")!=None:
            movie_data.append(i.find("font").contents[0])
        elif i.find("b")!=None:
            movie_dataappend(i.find("b").contents[0])

    ### Clean Data:

    # Remove extraneous tags
    movie_data = [a.contents[0] if type(a)!=bs4.element.NavigableString else a for a in movie_data]

    # Strip special characters
    movie_data = [re.sub("[^A-Za-z0-9-. ]+", "", a) for a in movie_data]

    # Fill NaNs
    movie_data = [np.nan if a =="na" else a for a in movie_data]

    # Set first 6 elements as column headers
    to_df = movie_data[6:]

    # Define the column names 
    columns = ["rank","title","studio","worldwide-gross","domestic-gross","domestic-pct","overseas-gross","overseas-pct"]

    # Convert to dataframe
    nrow = int(len(to_df)/len(columns)) 
    dirty_movies_df = pd.DataFrame(np.array(to_df).reshape(nrow,8),columns=columns)

    # Remove unnecessary columns
    dirty_movies_df = dirty_movies_df.iloc[: , 0:3]
    dirty_movies_df["rank"] = dirty_movies_df["rank"].apply(int)
    movies_df = dirty_movies_df.loc[dirty_movies_df["rank"] <=10,:]
    
    # Convert dataframe to list of dictionaries
    movie_dicts = movies_df.to_dict(orient="records") 
    
    print("Movies Scraped from BoxOfficeMojo.")
    
    return (movie_dicts) 

## FOR TESTING PURPOSES- REMOVE BEFORE FINAL PROJECT

In [31]:
x = movie_scraper()

Movies Scraped from BoxOfficeMojo.


In [32]:
print(x)

[{'rank': 1, 'title': 'Avengers Infinity War', 'studio': 'BV'}, {'rank': 2, 'title': 'Black Panther', 'studio': 'BV'}, {'rank': 3, 'title': 'Jurassic World Fallen Kingdom', 'studio': 'Uni.'}, {'rank': 4, 'title': 'Incredibles 2', 'studio': 'BV'}, {'rank': 5, 'title': 'Aquaman', 'studio': 'WB'}, {'rank': 6, 'title': 'Bohemian Rhapsody', 'studio': 'Fox'}, {'rank': 7, 'title': 'Venom 2018', 'studio': 'Sony'}, {'rank': 8, 'title': 'Mission Impossible - Fallout', 'studio': 'Par.'}, {'rank': 9, 'title': 'Deadpool 2', 'studio': 'Fox'}, {'rank': 10, 'title': 'Fantastic Beasts The Crimes of Grindelwald', 'studio': 'WB'}]


## END TESTING

## Scraping Billboard Music

In [54]:
def process_chart(data, year):
    
    """ Use the Python package for parsing HTML.  Calls and receives HTML as strings to process for artists."""
    
    # Create soup object to parse the html
    soup = bs(data,"html5lib")
    
    # Create a list to return
    list_albums = []

    # Inspect parsed html
    # For each article item, loop and identify tags to extract from.
    # For each entry, add a dictionary to the album list
    
    for item in soup.select("article"):
        rank = int(item.select_one(".ye-chart-item__rank").string.strip())
        title = item.select_one(".ye-chart-item__title").string.strip()
        artist = item.select_one(".ye-chart-item__artist").text.replace("\n", "")
        list_albums.append({"rank":rank, "title":title, "artist":artist,"year":year})
    
    return(list_albums)

In [55]:
def album_scraper():

    """Scrapes www.billboard.com for the top ten albums for 2008-2018 based on gross box-office amount.
    Returns a list of dictionaries with year, album title, and artist name"""
    
    year = str(2018)
    
    all_albums = []

    # Use requests library to get HTML
    url = requests.get("https://www.billboard.com/charts/year-end/"+str(year)+"/top-billboard-200-albums")
    
    # Parse content and create list of dictionaries using process_chart function
    data = url.content
    all_albums = process_chart(data,year)
    
    # Filter just the top 10 albums and insert into final list of dictionaries
    album_dicts = []
    for album in all_albums:
        if (album["rank"] < 11):
            album_dicts.append(album)
            
    print("Albums Scraped from Billboard.")
    
    return(album_dicts)

## FOR TESTING PURPOSES- REMOVE BEFORE FINAL PROJECT

In [35]:
x = album_scraper()

Albums Scraped from Billboard.


In [36]:
print(x)

[{'rank': 1, 'title': 'reputation', 'artist': 'Taylor Swift', 'year': '2018'}, {'rank': 2, 'title': 'Scorpion', 'artist': 'Drake', 'year': '2018'}, {'rank': 3, 'title': 'beerbongs & bentleys', 'artist': 'Post Malone', 'year': '2018'}, {'rank': 4, 'title': 'The Greatest Showman', 'artist': 'Soundtrack', 'year': '2018'}, {'rank': 5, 'title': '÷ (Divide)', 'artist': 'Ed Sheeran', 'year': '2018'}, {'rank': 6, 'title': 'Invasion Of Privacy', 'artist': 'Cardi B', 'year': '2018'}, {'rank': 7, 'title': 'ASTROWORLD', 'artist': 'Travis Scott', 'year': '2018'}, {'rank': 8, 'title': 'Stoney', 'artist': 'Post Malone', 'year': '2018'}, {'rank': 9, 'title': '?', 'artist': 'XXXTENTACION', 'year': '2018'}, {'rank': 10, 'title': 'Culture II', 'artist': 'Migos', 'year': '2018'}]


## END TESTING

## Scraping Metacritic

In [56]:
def metacritic_movie_scraper(url):

    """Scrapes given metacritic.com url for the movie review information.
    Returns a dictionary with number of user reviews, average user review, number of critic reviews, and critic score"""
    
    # Use splinter to get website information
    with Browser("chrome", **executable_path, headless=True) as browser:
        browser.visit(url)
        
        # Create a timestamp
        now = datetime.now()
        scrape_time = now.strftime("%Y-%m-%d %H:%M:%S")
        
        #Use beautiful soup to parse html
        soup = bs(browser.html, "html.parser")

    try:
        # Find number of reviews from users and critics
        rev_count_strings = soup.find_all("span", class_="based_on")
        user_rev_count = int(rev_count_strings[1].text.split(" ")[2])
        critic_rev_count = int(rev_count_strings[0].text.split(" ")[2])

        # Find review average from users and rating score from critics
        review_soup = soup.find_all("a", class_="metascore_anchor")
        user_rev_avg = float(review_soup[1].text)
        critic_rev_score = int(review_soup[0].text)

    # If page does note have review information, population review information with None values
    except (IndexError, AttributeError):
        user_rev_count = None
        critic_rev_count = None
        user_rev_avg = None
        critic_rev_score = None    
    
    # Return dictionary of book information
    movie_dict = {"user_rev_count": user_rev_count, "user_rev_avg": user_rev_avg, "critic_rev_count": critic_rev_count, "critic_rev_score": critic_rev_score, "scrape_time": scrape_time}
    return(movie_dict)

In [47]:
def metacritic_album_scraper(url):

    """Scrapes given metacritic.com url for the album review information.
    Returns a dictionary with number of user reviews, average user review, number of critic reviews, and critic score"""
    
    # Use splinter to get website information
    with Browser("chrome", **executable_path, headless=True) as browser:
        browser.visit(url)
        
        # Create a timestamp
        now = datetime.now()
        scrape_time = now.strftime("%Y-%m-%d %H:%M:%S")
        
        #Use beautiful soup to parse html
        soup = bs(browser.html, "html.parser")
    
    try:
        # Find review average from users and rating score from critics
        review_soup = soup.find_all("a", class_="metascore_anchor")
        user_rev_avg = float(review_soup[1].text)
        critic_rev_score = int(review_soup[0].text)

        # Find number of user reviews
        count_soup = soup.find("div",class_="module reviews_module user_reviews_module")
        user_rev_count_string = count_soup.find("span",class_="count")
        user_rev_count = int(user_rev_count_string.text)

        # Find number of critic reviews
        critic_rev_count_string = count_soup.find("span",class_="count")
        critic_rev_count = int(critic_rev_count_string.text)
    
    # If page does note have review information, population review information with None values
    except (IndexError, AttributeError) :
        user_rev_count = None
        critic_rev_count = None
        user_rev_avg = None
        critic_rev_score = None

    # Return dictionary of album information
    album_dict = {"user_rev_count": user_rev_count, "user_rev_avg": user_rev_avg, "critic_rev_count": critic_rev_count, "critic_rev_score": critic_rev_score, "scrape_time": scrape_time}
  
    return (album_dict)

In [57]:
def make_url_string(string):
    """Takes a string and returns a string to be inserted in url"""
    
    url_string = string.replace("(", "").replace(")","").replace("÷", "").replace("&", "").replace("-", "").\
    replace("  ", " ").replace(" ", "-").lower()
    
    if url_string.startswith("-"):
        url_string = url_string[1:]
    
    if url_string.endswith("-"):
        url_string = url_string[: -1]

    return(url_string)

## Create list of dictionaries for top movies and music

In [58]:
# Scrape BoxOfficeMojo and Billboard Music for a list of dictionaries of the top 10 movies for 2008-2018
movie_BOM_dicts = movie_scraper()
album_Bill_dicts = album_scraper()

Movies Scraped from BoxOfficeMojo.
Albums Scraped from Billboard.


## FOR TESTING PURPOSES- REMOVE BEFORE FINAL PROJECT

### Movie test

In [40]:
# change x to be 0-9
x = 1

In [41]:
movie = movie_BOM_dicts[x]

In [42]:
movie_dicts = []
# Create query url from dictionary values
movie_query = make_url_string(movie["title"])
movie_url = f"https://www.metacritic.com/movie/{movie_query}/details"

# Add review information to dictionary
movie_dicts.append({**movie, **metacritic_movie_scraper(movie_url)})
print(f"{movie['title']} scraped")

Black Panther scraped


In [43]:
print(movie_dicts)

[{'rank': 2, 'title': 'Black Panther', 'studio': 'BV', 'user_rev_count': 2829, 'user_rev_avg': 6.5, 'critic_rev_count': 55, 'critic_rev_score': 88, 'scrape_time': '2019-05-06 08:26:15'}]


In [ ]:
### ALBUM test

In [44]:
# change x to be 0-9
x = 1

In [45]:
album = album_Bill_dicts[x]

In [49]:
album_dicts = []
# Create query url from dictionary values
title_query = make_url_string(album["title"])
artist_query = make_url_string(album["artist"])
album_url = f"https://www.metacritic.com/music/{title_query}/{artist_query}"

# Add review information to dictionary
album_dicts.append({**album, **metacritic_album_scraper(album_url)})
print(f"{album['title']} scraped")

Scorpion scraped


In [50]:
print(album_dicts)

[{'rank': 2, 'title': 'Scorpion', 'artist': 'Drake', 'year': '2018', 'user_rev_count': 34, 'user_rev_avg': 3.8, 'critic_rev_count': 34, 'critic_rev_score': 67, 'scrape_time': '2019-05-06 08:29:02'}]


## END TESTING

In [59]:
# Add review information from Metacritic to new list of dictionaries for top movies
movie_dicts = []
for movie in movie_BOM_dicts:
    
    # Create query url from dictionary values
    movie_query = make_url_string(movie["title"])
    movie_url = f"https://www.metacritic.com/movie/{movie_query}/details"

    # Add review information to dictionary
    movie_dicts.append({**movie, **metacritic_movie_scraper(movie_url)})
    print(f"{movie['title']} scraped")
    
# Add review information from Metacritic to new list of dictionaries for top music albums
album_dicts = []
for album in album_Bill_dicts:
    # Create query url from dictionary values
    title_query = make_url_string(album["title"])
    artist_query = make_url_string(album["artist"])
    album_url = f"https://www.metacritic.com/music/{title_query}/{artist_query}"
    
    # Add review information to dictionary
    album_dicts.append({**album, **metacritic_album_scraper(album_url)})
    print(f"{album['title']} scraped")

Avengers Infinity War scraped
Black Panther scraped
Jurassic World Fallen Kingdom scraped
Incredibles 2 scraped
Aquaman scraped
Bohemian Rhapsody scraped
Venom 2018 scraped
Mission Impossible - Fallout scraped
Deadpool 2 scraped
Fantastic Beasts The Crimes of Grindelwald scraped
reputation scraped
Scorpion scraped
beerbongs & bentleys scraped
The Greatest Showman scraped
÷ (Divide) scraped
Invasion Of Privacy scraped
ASTROWORLD scraped
Stoney scraped
? scraped
Culture II scraped


## Populate mongo database

In [60]:
# Connect to mongo using pymongo to create local database
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Create Top 10 database
db = client.top_10_db

# Create movies and albums collections
movies = db.movies
albums = db.albums

# Insert top 10 movies and albums for 2008-2018
# GRETEL - FIGURE OUT WHETHER WE WANT TO UPSERT
db.movies.insert_many(movie_dicts)
db.albums.insert_many(album_dicts)

## Testing